In [269]:
import os
os.environ['OMP_NUM_THREADS'] = '1'
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [270]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv("Resources/crypto_market_data.csv",
                             index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [271]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [272]:
# Plot data to see what's in the DataFrame

df_market_data.hvplot.line(
    width = 800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

# Prepare the Data (Most Important Step)

In [273]:
# Create an instance of the StandardScaler
scaler = StandardScaler()

# Normalize the data
scaled_data = scaler.fit_transform(df_market_data)

# Convert the normalized data back to a DataFrame
df_market_data_scaled = pd.DataFrame(scaled_data, columns=df_market_data.columns)

# Display the normalized data
df_market_data_scaled.head()


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
0,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
1,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
2,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
3,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
4,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


In [274]:
df_market_data_scaled = pd.DataFrame(scaled_data, columns=df_market_data.columns)

# Copy the crypto names from the original data
df_market_data_scaled['coin_id'] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled.set_index('coin_id', inplace=True)

# Display sample data
df_market_data_scaled.head()


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


Best Value for k Using the Original Data.

In [275]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12)) # Stops at 11, does not include 12
k_values

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [276]:
# Create an empty list to store the inertia values
inert_values = []

# Create a for loop to compute the inertia with each possible value of k
for k in k_values:
    # Create a KMeans model using the loop counter for the n_clusters
    kmeans_model = KMeans(n_clusters=k, n_init=10)
    
    # Fit the model to the data using `df_market_data_scaled`
    kmeans_model.fit(df_market_data_scaled)
    
    # Append the model.inertia_ to the inertia list
    inert_values.append(kmeans_model.inertia_)


In [277]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    'k_values': k_values,
    'inert_values': inert_values
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_1 = pd.DataFrame(elbow_data)


In [278]:
df_elbow_1.hvplot.line(x='k_values', y='inert_values', xlabel='Number of Clusters (k)', ylabel='Inertia', title='Elbow Curve', xticks=list(range(1, len(df_elbow_1))))



:Curve   [k_values]   (inert_values)

The elbow seems to occur at k=3, where the curve starts to flatten out more significantly. This suggests that the best value for k in the k-means clustering algorithm for this dataset would be 3. Choosing k=3 should provide a reasonable segmentation of the data into clusters without introducing unnecessary complexity or overfitting.

### Cluster Cryptocurrencies with K-Means Using the Original Data

In [279]:
# Initialize the K-Means model using the best value for k
kmeans_model = KMeans(n_clusters=3)

In [280]:
# Fit the K-Means model using the scaled data
kmeans_model.fit(df_market_data_scaled)


KMeans(n_clusters=3)

In [281]:
# Predict the clusters to group the cryptocurrencies using the scaled data
cluster_labels = kmeans_model.predict(df_market_data_scaled)

# Print the resulting array of cluster values
print(cluster_labels)


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 0
 1 1 1 1]


In [282]:
# Create a copy of the DataFrame
df_copy = df_market_data_scaled.copy()


In [283]:
# Add a new column to the DataFrame with the predicted clusters
df_copy['Cluster'] = cluster_labels

# Display sample data
df_copy.head()


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,Cluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,1
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,1
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,1
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,1
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,1


In [284]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']  # These are  hex color codes for blue, orange, and


df_copy.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="Cluster",  # Ensure this parameter is used to group by clusters
    color=colors,
    hover_cols=["coin_id"]
)


:NdOverlay   [Cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

### Optimize Clusters with Principal Component Analysis.

In [285]:
# Create a PCA model instance and set `n_components=3`
pca_model = PCA(n_components=3)


In [286]:
# Use the PCA model with `fit_transform` to reduce to three principal components
pca_data = pca_model.fit_transform(df_market_data_scaled)

# Convert the PCA data to a DataFrame
df_pca = pd.DataFrame(pca_data, columns=['PC1', 'PC2', 'PC3'])

# View the first five rows of the DataFrame
df_pca.head()


,PC1,PC2,PC3
0,-0.600667,0.842760,0.461595
1,-0.458261,0.458466,0.952877
2,-0.433070,-0.168126,-0.641752
3,-0.471835,-0.222660,-0.479053
4,-1.157800,2.041209,1.859715


In [287]:
# Retrieve the explained variance to determine how much information 
explained_variance = pca_model.explained_variance_ratio_
total_explained_variance = sum(explained_variance)

total_explained_variance_percent = total_explained_variance * 100

print(f'Total Variance in percentage: {total_explained_variance_percent}%')
print(f'Total Variance in float: {total_explained_variance}')


Total Variance in percentage: 89.5031657030984%
Total Variance in float: 0.895031657030984


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 89.5031657030984% or 0.895031657030984

In [288]:
# Create a new DataFrame with the PCA data
df_pca_data = pd.DataFrame(pca_data, columns=['PC1', 'PC2', 'PC3'])

# Copy the crypto names from the original data
df_pca_data['coin_id'] = df_market_data.index

# Set the coinid column as index
df_pca_data.set_index('coin_id', inplace=True)

# Display sample data
df_pca_data.head()


,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


### Find the Best Value for k Using the PCA Data

In [289]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12)) # Stops at 11, does not include 12
k_values

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [290]:
# Create an empty list to store the inertia values
inert_values = []

# Create a for loop to compute the inertia with each possible value of k
for k in k_values:
    # Create a KMeans model using the loop counter for the n_clusters
    kmeans_model = KMeans(n_clusters=k, n_init=10)
    
    # Fit the model to the data using `df_market_data_pca`
    kmeans_model.fit(df_pca_data)
    
    # Append the model.inertia_ to the inertia list
    inert_values.append(kmeans_model.inertia_)


In [291]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    'k_values': k_values,
    'inert_values': inert_values
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_2 = pd.DataFrame(elbow_data)


In [292]:
df_elbow_2.hvplot.line(x='k_values', y='inert_values', xlabel='Number of Clusters (k)', ylabel='Inertia', title='Elbow Curve', xticks=list(range(1, len(df_elbow_2))))


:Curve   [k_values]   (inert_values)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** There seems to be a slight bend around k=4. After that point, the rate of decrease in WCSS slows down, suggesting that increasing the number of clusters beyond 4 results in diminishing returns in terms of variance explained. Therefore, k=4 could be considered as the best value for k when using the PCA data.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** Yes, it differs. The best k value suggested by the Elbow Curve using the PCA data appears to be k=4, while the best k value identified using the original data was k=3. This discrepancy can occur because PCA reduces dimensionality by combining features, which can alter the distribution of data points and potentially reveal different clustering structures. As a result, the optimal number of clusters (k) might change when using PCA-transformed data compared to the original data.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [293]:
# Initialize the K-Means model using the best value for k
kmeans_model = KMeans(n_clusters=4)


In [294]:
# Fit the K-Means model using the PCA data
kmeans_model.fit(df_pca_data)

KMeans(n_clusters=4)

In [295]:
# Predict the clusters to group the cryptocurrencies using the PCA data

cluster_labels_pca = kmeans_model.predict(df_pca_data)

# Print the resulting array of cluster values
print(cluster_labels_pca)


[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 1 2 0 0 3
 0 0 0 0]


In [296]:
# Create a copy of the DataFrame with the PCA data
df_pca_copy = df_pca_data.copy()

# Add a new column to the DataFrame with the predicted clusters
df_pca_copy['Cluster'] = cluster_labels_pca

# Display sample data
df_pca_copy.head()


,PC1,PC2,PC3,Cluster
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,2
ethereum,-0.458261,0.458466,0.952877,2
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,2


In [297]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.


df_pca_copy.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="Cluster",
    hover_cols=["coin_id"]
)


:NdOverlay   [Cluster]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, we will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [302]:
# Create the Elbow curve plot 1
elbow_curve1 = df_elbow_1.hvplot.line(
    x='k_values',
    y='inert_values',
    xlabel='Number of Clusters (k)',
    ylabel='Inertia',
    title='Elbow Curve for Original Data'
).opts(xticks=list(range(1, len(df_elbow_1) + 1)))

# Create the Elbow curve plot 2
elbow_curve2 = df_elbow_2.hvplot.line(
    x='k_values',
    y='inert_values',
    xlabel='Number of Clusters (k)',
    ylabel='Inertia',
    title='Elbow Curve for PCA Data'
).opts(xticks=list(range(1, len(df_elbow_2) + 1)))

# Combine the plots side by side
composite_plot = (elbow_curve1 + elbow_curve2).cols(1)

# Display the composite plot
composite_plot


:Layout
   .Curve.I  :Curve   [k_values]   (inert_values)
   .Curve.II :Curve   [k_values]   (inert_values)

In [303]:
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']  # These are hex color codes for blue, orange, green, and red

# Scatter plot for original data with a title
scatter_plot1 = df_copy.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="Cluster",
    color=colors,
    hover_cols=["coin_id"],
    title="Original Data: 24h vs 7d Price Change"
)

# Scatter plot for PCA data with a title
scatter_plot2 = df_pca_copy.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="Cluster",
    color=colors,
    hover_cols=["coin_id"],
    title="PCA Data: PC1 vs PC2"
)

# Combine the scatter plots side by side (horizontally)
composite_plot = (scatter_plot1 + scatter_plot2).cols(2)

# Display the composite plot
composite_plot


:Layout
   .NdOverlay.I  :NdOverlay   [Cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [Cluster]
      :Scatter   [PC1]   (PC2,coin_id)